# 1 PDF转化
由于提取代码使用PDF提取效果较好，使用topdf.py将ppt等文件转化为PDF即可，使用过程中仅需输入仅需要输入文件夹路径即可，代码如下。

### 使用前pip install os comtypes

In [14]:
import os
from comtypes.client import CreateObject

def convert_to_pdf(input_path, output_path, file_type):
    try:
        if file_type in ['.docx', '.doc']:
            word = CreateObject('Word.Application')
            doc = word.Documents.Open(input_path)
            doc.SaveAs(output_path, FileFormat=17)  # FileFormat=17 corresponds to PDF
            doc.Close()
            word.Quit()
        elif file_type in ['.pptx', '.ppt']:
            powerpoint = CreateObject('PowerPoint.Application')
            presentation = powerpoint.Presentations.Open(input_path)
            presentation.SaveAs(output_path, 32)  # 32 corresponds to PDF
            presentation.Close()
            powerpoint.Quit()
    except Exception as e:
        print(f"Error converting {input_path}: {e}")

def batch_convert(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            if ext in ['.docx', '.doc', '.pptx', '.ppt']:
                input_file = os.path.join(root, file)
                output_file = os.path.splitext(input_file)[0] + '.pdf'
                print(f"Converting: {input_file} -> {output_file}")
                convert_to_pdf(input_file, output_file, ext)

# 替换为你的文件夹路径
folder_path = r"G:\yuliaoku\新能源减排服务"
batch_convert(folder_path)

# 2 PDF读取
使用OPNEDATALAB的开源项目MinerU，使用pdf-3.2.py进行PDF提取转化成markdowm格式，使用过程仅需要输入pdf的存放文件夹路径即可，代码如下

## 使用前注意事项
访问其开源网页https://mineru.readthedocs.io/zh-cn/latest/user_guide/install/install.html


### 创建虚拟环境

In [ ]:
conda create -n MinerU python=3.10
conda activate MinerU
pip install -U magic-pdf[full] --extra-index-url https://wheels.myhloli.com -i https://mirrors.aliyun.com/pypi/simple

### 下载权重文件

In [ ]:
pip install huggingface_hub
wget https://gitee.com/myhloli/MinerU/raw/master/scripts/download_models_hf.py -O download_models_hf.py
python download_models_hf.py

### 使用CUDA加速
建议使用cuda加速，显存最低需要8g以上
完成下载权重文件步骤后，脚本会自动生成用户目录下的magic-pdf.json文件，并自动配置默认模型路径。您可在【用户目录】下找到magic-pdf.json文件。

## 问题存在
MinerU库中的配置文件不支持多卡去跑，目前只能单卡

In [ ]:
import os
from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
from magic_pdf.data.dataset import PymuDocDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
from magic_pdf.config.enums import SupportedPdfParseMethod

def is_file_processed(pdf_file_name, output_dir):
    """
    检查 PDF 文件是否已经被处理。
    Args:
        pdf_file_name (str): PDF 文件路径。
        output_dir (str): 输出文件夹路径。
    Returns:
        bool: 如果已处理则返回 True，否则返回 False。
    """
    name_without_suff = os.path.splitext(os.path.basename(pdf_file_name))[0]
    md_file_path = os.path.join(output_dir, "markdown", f"{name_without_suff}.md")
    content_list_path = os.path.join(output_dir, "markdown", f"{name_without_suff}_content_list.json")

    return os.path.exists(md_file_path) and os.path.exists(content_list_path)

def process_pdf(pdf_file_name, output_dir="output"):
    """
    处理单个 PDF 文件，支持 OCR 模式和文本模式。
    Args:
        pdf_file_name (str): 输入的 PDF 文件路径。
        output_dir (str): 输出文件夹路径。
    """
    try:
        # 检查文件是否已经被处理
        if is_file_processed(pdf_file_name, output_dir):
            print(f"Skipping already processed file: {pdf_file_name}")
            return

        # 获取文件名（无后缀）
        name_without_suff = os.path.splitext(os.path.basename(pdf_file_name))[0]

        # 创建输出目录
        local_image_dir = os.path.join(output_dir, "images", os.path.splitext(os.path.basename(pdf_file_name))[0])
        local_md_dir = os.path.join(output_dir, "markdown")
        os.makedirs(local_image_dir, exist_ok=True)
        os.makedirs(local_md_dir, exist_ok=True)

        # 初始化数据写入器
        image_writer = FileBasedDataWriter(local_image_dir)
        md_writer = FileBasedDataWriter(local_md_dir)

        # 读取 PDF 文件的字节流
        reader = FileBasedDataReader("")
        pdf_bytes = reader.read(pdf_file_name)

        # 创建数据集实例
        ds = PymuDocDataset(pdf_bytes)

        # 推断并分类
        if ds.classify() == SupportedPdfParseMethod.OCR:
            infer_result = ds.apply(doc_analyze, ocr=True)
            pipe_result = infer_result.pipe_ocr_mode(image_writer)
        else:
            infer_result = ds.apply(doc_analyze, ocr=False)
            pipe_result = infer_result.pipe_txt_mode(image_writer)

        # 输出结果文件
        pipe_result.dump_md(md_writer, f"{name_without_suff}.md", os.path.basename(local_image_dir))
        pipe_result.dump_content_list(md_writer, f"{name_without_suff}_content_list.json", os.path.basename(local_image_dir))

        print(f"Successfully processed: {pdf_file_name}")

    except Exception as e:
        print(f"Error processing {pdf_file_name}: {e}")

def process_folder(input_folder, output_dir="output"):
    """
    批量处理文件夹中的所有 PDF 文件。
    Args:
        input_folder (str): 输入文件夹路径。
        output_dir (str): 输出文件夹路径。
    """
    os.makedirs(output_dir, exist_ok=True)
    skipped_files = []
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_file_path = os.path.join(root, file)
                if is_file_processed(pdf_file_path, output_dir):
                    skipped_files.append(pdf_file_path)
                process_pdf(pdf_file_path, output_dir)
    
    if skipped_files:
        print("\nSkipped files:")
        for skipped_file in skipped_files:
            print(skipped_file)

if __name__ == "__main__":
    # 输入文件夹和输出文件夹路径
    input_folder = "/home/yancong/xin-llm/第二次拷贝/books"
    output_dir = "/home/yancong/xin-llm/第二次拷贝/books/output"

    # 批量处理整个文件夹中的 PDF 文件
    process_folder(input_folder, output_dir)


### 已修改
使用duo.py可以使用多卡去跑

In [ ]:
import os
import math
import torch.multiprocessing as mp
from multiprocessing import Pool

def is_file_processed(pdf_file_name, output_dir):
    """
    检查 PDF 文件是否已经被处理。
    """
    name_without_suff = os.path.splitext(os.path.basename(pdf_file_name))[0]
    md_file_path = os.path.join(output_dir, "markdown", f"{name_without_suff}.md")
    content_list_path = os.path.join(output_dir, "markdown", f"{name_without_suff}_content_list.json")
    return os.path.exists(md_file_path) and os.path.exists(content_list_path)

def process_pdf(pdf_file_name, output_dir="output"):
    """
    处理单个 PDF 文件，支持 OCR 模式和文本模式。
    注意：不要在此函数外部 import torch 或 magic_pdf，而是在此函数内部 import，
    这样可以确保子进程已设置好 CUDA_VISIBLE_DEVICES 后再初始化相关库。
    """
    # 只有当我们真正执行到这里时，才 import torch 和 magic_pdf
    import torch
    from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
    from magic_pdf.data.dataset import PymuDocDataset
    from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
    from magic_pdf.config.enums import SupportedPdfParseMethod

    try:
        if is_file_processed(pdf_file_name, output_dir):
            print(f"Skipping already processed file: {pdf_file_name}")
            return

        name_without_suff = os.path.splitext(os.path.basename(pdf_file_name))[0]

        local_image_dir = os.path.join(output_dir, "images", name_without_suff)
        local_md_dir = os.path.join(output_dir, "markdown")
        os.makedirs(local_image_dir, exist_ok=True)
        os.makedirs(local_md_dir, exist_ok=True)

        image_writer = FileBasedDataWriter(local_image_dir)
        md_writer = FileBasedDataWriter(local_md_dir)

        reader = FileBasedDataReader("")
        pdf_bytes = reader.read(pdf_file_name)
        ds = PymuDocDataset(pdf_bytes)

        # 如果 doc_analyze 内部会用到 GPU（例如 model.to("cuda:0")），
        # 那么此时进程只看到一块卡，它的 index 就是 0。
        # 所以 "cuda:0" 会对应系统的物理 GPU #gpu_id。
        # 这里无需改动 doc_analyze 的代码，但如果 doc_analyze 里硬编码了 "cuda:0"
        # 作为全局物理卡，就需要改成对子进程而言的 "cuda:0"。

        # 推断并分类
        parse_method = ds.classify()
        if parse_method == SupportedPdfParseMethod.OCR:
            infer_result = ds.apply(doc_analyze, ocr=True)
            pipe_result = infer_result.pipe_ocr_mode(image_writer)
        else:
            infer_result = ds.apply(doc_analyze, ocr=False)
            pipe_result = infer_result.pipe_txt_mode(image_writer)

        # 输出结果文件
        pipe_result.dump_md(md_writer, f"{name_without_suff}.md", os.path.basename(local_image_dir))
        pipe_result.dump_content_list(md_writer, f"{name_without_suff}_content_list.json", os.path.basename(local_image_dir))

        print(f"Successfully processed: {pdf_file_name}")

    except Exception as e:
        print(f"Error processing {pdf_file_name}: {e}")

def gather_pdf_files(input_folder):
    """
    扫描文件夹，返回所有 PDF 文件的绝对路径列表。
    """
    pdf_files = []
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_file_path = os.path.join(root, file)
                pdf_files.append(pdf_file_path)
    return pdf_files

def process_chunk(pdf_files_chunk, output_dir, gpu_id):
    """
    子进程函数：
    - 先设置子进程可见的 GPU
    - 再 import torch (以及 doc_analyze 相关的库) 或在 process_pdf 里再 import
    - 然后遍历要处理的 PDF 文件
    """

    # 第一步：告诉当前子进程，只能使用物理 GPU #gpu_id
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

    # 第二步：现在才可以 import torch（如需在此函数用 GPU）
    import torch

    print(f"\n[Process {os.getpid()}] I will use physical GPU={gpu_id}.")
    print(f"  Before any CUDA ops: torch.cuda.device_count() = {torch.cuda.device_count()}")

    # 如果 torch.cuda 还未初始化，这时 device_count() 可能就是 1，或者仍然是4（取决于实际情况）。
    # 要进一步确认可以尝试显式初始化，比如分配一个张量：

    if torch.cuda.is_available():
        try:
            test_x = torch.randn(1).cuda()
            print("  GPU test allocation successful.")
        except Exception as e:
            print(f"  GPU test allocation failed: {e}")

    # 第三步：处理该 chunk 下的所有 PDF 文件
    for pdf_file in pdf_files_chunk:
        process_pdf(pdf_file, output_dir)

    print(f"[Process {os.getpid()}] Done with GPU {gpu_id}.\n")

def process_folder(input_folder, output_dir="output", num_gpus=4):
    """
    主调度函数：
    1. 收集所有 PDF 文件
    2. 按 num_gpus 分成若干个块
    3. 多进程并行，每个进程分配到一个 GPU
    """
    os.makedirs(output_dir, exist_ok=True)

    all_pdf_files = gather_pdf_files(input_folder)
    total_files = len(all_pdf_files)
    if total_files == 0:
        print("No PDF files found.")
        return

    print(f"Found {total_files} PDF files in '{input_folder}'. Starting processing...")

    # 将 PDF 文件列表拆分为 num_gpus 份
    chunk_size = math.ceil(total_files / num_gpus)
    pdf_file_chunks = [
        all_pdf_files[i : i + chunk_size]
        for i in range(0, total_files, chunk_size)
    ]

    # 创建多进程池并开始并行处理
    pool = Pool(processes=num_gpus)
    for gpu_id, chunk in enumerate(pdf_file_chunks):
        pool.apply_async(process_chunk, (chunk, output_dir, gpu_id))

    pool.close()
    pool.join()

    print("All processes are done.")

if __name__ == "__main__":
    # 第一步：使用 'spawn' 模式启动子进程，以避免在 Linux 下使用 'fork' 导致的 CUDA 上下文继承问题
    mp.set_start_method('spawn', force=True)

    # 你要处理的 PDF 所在文件夹
    input_folder = "/home/yancong/第二次语料库/books/化工设备设计全书/"
    output_dir   = "/home/yancong/第二次语料库/books/化工设备设计全书/output"

    # 同时使用 4 张 GPU
    process_folder(input_folder, output_dir, num_gpus=4)


# 文本清洗
清洗markdown内容并将其转化为txt格式文本，包括降噪以及删除无关内容等功能，后期可以根据要求添加功能
使用txtclean（1）.py进行工作
使用过程中仅需输入pdf提取过程的输出文件夹即可


In [ ]:
import os
import re


def clean_markdown(content):
    """
    清洗 Markdown 内容
    - 删除图片和链接标记
    - 移除无关元数据（如版权信息、邮箱等）
    - 删除目录部分（通过连续的章节标题判断）
    - 删除多余空行和尾部空格
    - 删除方括号及其内容，以及参考文献
    - 删除类似 Bopp(1984), Brown（1983） 的参考文献
    """
    cleaned_content = []
    is_in_toc = False  # 标记是否处于目录部分
    previous_line = None  # 用于追踪上一行是否为空

    for line in content:
        # 去除行尾空格
        line = line.rstrip()

        # 检查是否进入目录部分
        if line.lower().startswith("contents") or "目录" in line:
            is_in_toc = True
            continue
        # 检查是否结束目录部分（假设目录以空行或某些章节开始）
        if is_in_toc and (not line.strip() or line.startswith("#")):
            is_in_toc = False
        # 跳过目录部分
        if is_in_toc:
            continue

        # 跳过包含图片和链接的行
        if "![" in line or "](" in line:
            continue
        # 跳过无关的元数据行，如版权、邮箱、网址等
        if "copyright" in line.lower() or "email" in line.lower() or "www." in line.lower():
            continue

        # 删除方括号及其内容
        line = re.sub(r"\[.*?\]", "", line)

        # 删除类似 "Bopp(1984)", "Brown（1983）", "Brown和Phillips（1989，1991）" 的内容
        line = re.sub(r"\b[A-Za-z\u4e00-\u9fa5]+(?:和[A-Za-z\u4e00-\u9fa5]+)?\s*[（(]\d{4}(?:，\d{4})*[）)]", "", line)

        # 删除多余的空行（连续多个空行只保留一个）
        if not line.strip():
            if previous_line is None or not previous_line.strip():
                continue  # 当前行和上一行都是空行时跳过
            else:
                cleaned_content.append("")  # 保留一个空行
        else:
            cleaned_content.append(line)  # 添加非空行

        # 更新上一行内容
        previous_line = line

    return cleaned_content

def clean_and_extract_markdown(content):
    """
    提取 Markdown 中标题和其下的文本，并清洗标题和正文
    - 提取标题及其下的文本
    - 删除标题及其下的文本小于等于 20 个字符的部分
    - 删除指定标题（如“前言”）和包含关键词（如“思考题”）的段落
    - 将文本拼接成长段格式，不换行
    """
    cleaned_content = clean_markdown(content)  # 先执行基本清洗
    result = []
    current_title = None
    current_text = []
    skip_section = False  # 标记是否跳过当前段落

    for line in cleaned_content:
        # 判断是否是标题（假设标题为 Markdown 标题格式，如 # 或 ## 开头）
        if line.startswith("#"):
            # 如果已有标题和文本，进行处理
            if current_title and current_text:
                combined_text = "".join(current_text).strip()
                # 检查标题下的文本长度是否超过 20
                if not skip_section and len(re.findall(r"[\u4e00-\u9fa5\w]+", combined_text)) > 20:
                    result.append(f"{current_title}\n{combined_text}\n")

            # 更新当前标题，清空文本，并检查是否需要跳过
            current_title = line.strip()
            current_text = []
            skip_section = any(keyword in current_title for keyword in ["前言", "思考题", "参考文献"])
        else:
            # 累积当前标题下的文本
            if not skip_section and line.strip():  # 忽略空行
                current_text.append(line.strip())

    # 处理最后一个标题和文本
    if current_title and current_text:
        combined_text = "".join(current_text).strip()
        if not skip_section and len(re.findall(r"[\u4e00-\u9fa5\w]+", combined_text)) > 20:
            result.append(f"{current_title}\n{combined_text}\n")

    return result


def process_markdown_files(input_folder, output_folder):
    """
    处理文件夹内的所有 Markdown 文件并保存成 .txt 文件
    - 读取每个 Markdown 文件
    - 清洗并提取标题及其下的文本
    - 清洗后保存为 .txt 文件
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, filename)
        if os.path.isfile(input_file_path) and filename.endswith('.md'):
            with open(input_file_path, 'r', encoding='utf-8') as file:
                content = file.readlines()

            # 清洗并提取内容
            cleaned_content = clean_and_extract_markdown(content)

            # 输出结果到目标文件夹
            if cleaned_content:  # 只有当内容非空时才保存文件
                output_file_path = os.path.join(output_folder, filename.replace('.md', '.txt'))
                with open(output_file_path, 'w', encoding='utf-8') as cleaned_file:
                    cleaned_file.write("\n".join(cleaned_content))
                print(f"Processed and saved: {output_file_path}")
            else:
                print(f"Skipped empty result for: {input_file_path}")


# 示例：读取 input_folder 内的文件并输出到 output_folder
input_folder = r"G:\yuliaoku\新能源热利用与热发电原理及系统\output\markdown"
output_folder = r"G:\yuliaoku\新能源热利用与热发电原理及系统\cleaned_markdown_files_txt"

process_markdown_files(input_folder, output_folder)


# 文本合并及JSONL的转化

## 文本合并
使用me.py进行文本合并，输入路径为txt清理的输出路径

In [ ]:
import os

# 设置输入文件夹和输出文件路径
input_folder = r"G:\yuliaoku\1-quan\课程" # 替换为您的文件夹路径
output_file = r"G:\yuliaoku\1-quan\课程/未去重.txt"  # 替换为合并后的文件路径

# 创建或清空输出文件
with open(output_file, "w", encoding="utf-8") as outfile:
    # 遍历文件夹中的所有 .txt 文件
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):
            file_path = os.path.join(input_folder, file_name)
            print(f"正在处理文件: {file_path}")
            # 读取每个文件并写入输出文件
            with open(file_path, "r", encoding="utf-8") as infile:
                outfile.write(infile.read())
                outfile.write("\n")  # 添加换行符以分隔文件内容

print(f"所有 .txt 文件已合并到 {output_file}")

## jsonl的转化
使用jsonl.py进行转化，输入文件为me.py合并的文件

In [ ]:
import os
import json


def txt_to_jsonl(input_folder, output_folder):
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)

    # 遍历输入文件夹中的所有txt文件
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):
            input_file_path = os.path.join(input_folder, file_name)
            output_file_name = file_name.replace(".txt", ".jsonl")
            output_file_path = os.path.join(output_folder, output_file_name)

            with open(input_file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            jsonl_data = []
            section_title = None
            content = []

            for line in lines:
                if line.startswith("#"):  # 标题行
                    if section_title:
                        jsonl_data.append({"section": section_title, "content": "".join(content).strip()})
                    section_title = line.strip("#").strip()
                    content = []
                else:
                    content.append(line)

            # 添加最后的部分
            if section_title:
                jsonl_data.append({"section": section_title, "content": "".join(content).strip()})

            with open(output_file_path, 'w', encoding='utf-8') as f:
                for entry in jsonl_data:
                    f.write(json.dumps(entry, ensure_ascii=False) + "\n")
            print(f"Converted: {file_name} -> {output_file_name}")


# 设置输入和输出文件夹路径
input_folder = r"G:\yuliaoku\jsonl" # 替换为您的输入文件夹路径
output_folder =r"G:\yuliaoku\jsonl"  # 替换为您的输出文件夹路径

# 执行转换
txt_to_jsonl(input_folder, output_folder)

# 文本去重
根据sentence-bret模型根据语义去重，可以调整similarity_threshold=0.8的值来设置去重力度，越接近0去重力度越大

In [ ]:

import sys
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm


def semantic_deduplicate(input_file, output_file, model_name='sentence-transformers/all-MiniLM-L6-v2',
                         similarity_threshold=0.9, batch_size=1024, encoding='utf-8'):
    """
    使用Sentence-BERT模型对文本进行语义去重的基础示例。

    参数说明：
    - input_file: 输入文本文件，每行一条文本记录
    - output_file: 输出去重后的文本文件
    - model_name: SentenceTransformer可加载的模型名称
    - similarity_threshold: 相似度阈值（0~1之间的余弦相似度）
    - batch_size: 批处理大小，每次计算多少行的嵌入
    - encoding: 文件编码

    步骤：
    1. 批量读取文本行并计算嵌入向量
    2. 对每行文本向量与已保留向量集计算相似度，若最高相似度低于阈值则保留该行并添加其向量到集合中。
    """
    model = SentenceTransformer(model_name)

    # 用于保存已保留行的向量
    retained_embeddings = []
    retained_texts = []

    with open(input_file, 'r', encoding=encoding) as fin, \
            open(output_file, 'w', encoding=encoding) as fout:

        buffer_lines = []
        line_count = 0
        unique_count = 0

        for line in tqdm(fin, desc="Processing"):
            text = line.strip()
            if not text:
                continue
            buffer_lines.append(text)

            # 批处理，当积累到一定数量时才计算嵌入
            if len(buffer_lines) >= batch_size:
                unique_lines = process_batch(buffer_lines, model, retained_embeddings, retained_texts,
                                             similarity_threshold)
                # 将去重后保留的行写入文件，并更新retained_embeddings
                for t, emb in unique_lines:
                    fout.write(t + "\n")
                    retained_embeddings.append(emb)
                    retained_texts.append(t)
                unique_count += len(unique_lines)
                line_count += len(buffer_lines)
                buffer_lines = []

        # 处理剩余行
        if buffer_lines:
            unique_lines = process_batch(buffer_lines, model, retained_embeddings, retained_texts, similarity_threshold)
            for t, emb in unique_lines:
                fout.write(t + "\n")
                retained_embeddings.append(emb)
                retained_texts.append(t)
            unique_count += len(unique_lines)
            line_count += len(buffer_lines)

        print(f"处理完成！共处理 {line_count} 行，最终输出 {unique_count} 行。", file=sys.stderr)


def process_batch(lines, model, retained_embeddings, retained_texts, similarity_threshold):
    """
    对一批文本行进行嵌入计算，然后对每行查找与已保留文本的最高相似度，决定是否保留。
    返回 (text, embedding) 的列表。
    """
    if len(lines) == 0:
        return []

    embs = model.encode(lines, convert_to_numpy=True)
    # 归一化，使向量长度为1
    embs = embs / np.linalg.norm(embs, axis=1, keepdims=True)

    results = []
    # 若尚无已保留文本，全部保留
    if len(retained_embeddings) == 0:
        for i, line in enumerate(lines):
            results.append((line, embs[i]))
        return results

    # 将已保留的嵌入合并为矩阵
    retained_matrix = np.vstack(retained_embeddings) if len(retained_embeddings) > 0 else None
    for i, line in enumerate(lines):
        emb = embs[i]
        # 计算与已保留嵌入的相似度（点积即为余弦相似度，因为已归一化）
        scores = np.dot(retained_matrix, emb)
        max_score = np.max(scores) if len(scores) > 0 else 0
        if max_score < similarity_threshold:
            # 没有相似度超过阈值，保留该文本
            results.append((line, emb))
        # 否则不保留

    return results


if __name__ == '__main__':
    input_path = r"G:\yuliaoku\1-quan\课程/未去重.txt"
    output_path =  r"G:\yuliaoku\1-quan\课程/去重.txt"
    semantic_deduplicate(input_path, output_path, similarity_threshold=0.8)
